In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep

In [2]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36'}

s = requests.Session()

response = s.get("https://www.transfermarkt.com/", headers=headers)

content = response.content

site = BeautifulSoup(content, 'html.parser')

In [3]:
def open(link):
    response = s.get(link, headers=headers)
    content = response.content
    site = BeautifulSoup(content, 'html.parser')
    return site

In [4]:
def get_stats_data(link):
    response = s.get(link, headers=headers)
    content = response.content
    site = BeautifulSoup(content, 'html.parser')
    
    second_link = site.find("a", class_="tm-subnav-item megamenu")['href']
    
    name = site.find("h1", class_='data-header__headline-wrapper data-header__headline-wrapper--oswald').text.strip()
    matches = site.find("tfoot").find_all("td", class_="zentriert")[0].text
    wins = site.find("tfoot").find_all("td", class_="zentriert")[1].text
    draws = site.find("tfoot").find_all("td", class_="zentriert")[2].text
    losses = site.find("tfoot").find_all("td", class_="zentriert")[3].text
    pts = site.find("tfoot").find_all("td", class_="zentriert")[4].text
    goals = site.find("tfoot").find_all("td", class_="zentriert")[5].text
    spectators = site.find("tfoot").find("td", class_="rechts").text
    
    columns = [name, matches, wins, draws, losses, pts, goals, spectators]
    c =['name', 'matches', 'wins', 'draws', 'losses', 'pts', 'goals', 'spectators']
    #columns = list(zip([name], [matches], [wins], [draws], [losses], [pts], [goals], [spectators]))
    #df = pd.DataFrame(columns,
    #           columns =['name', 'matches', 'wins', 'draws', 'losses', 'pts', 'goals', 'spectators'])
    #return df
    return columns, c

In [5]:
def get_squad_data(link):
    response = s.get(link, headers=headers)
    content = response.content
    site = BeautifulSoup(content, 'html.parser')
    
    rows = site.find_all("div", class_='box')[1].find('tbody').find_all('tr')
    total_row = site.find_all("div", class_='box')[1].find('tfoot').find('tr')
    
    oages = [row.find("td", class_="zentriert").text for row in rows]
    gk_oage = oages[0] if len(oages)> 0 else -1
    defense_oage = oages[1] if len(oages)> 1 else -1
    attack_oage = oages[2] if len(oages)> 2 else -1
    mkt_values = [row.find_all("td", class_="rechts")[0].text for row in rows]
    gk_mkt = mkt_values[0] if len(mkt_values)> 0 else -1
    defense_mkt = mkt_values[1] if len(mkt_values)> 1 else -1
    attack_mkt = mkt_values[2] if len(mkt_values)> 2 else -1
    omkt_values = [row.find_all("td", class_="rechts")[1].text for row in rows]
    gk_omkt = omkt_values[0] if len(omkt_values)> 0 else -1
    defense_omkt = omkt_values[1] if len(omkt_values)> 1 else -1
    attack_omkt = omkt_values[2] if len(omkt_values)> 2 else -1

    total_oage = total_row.find("td", class_="zentriert").text if total_row.find("td", class_="zentriert") is not None else -1
    tlist = total_row.find_all("td", class_="rechts")
    total_mkt = tlist[1].text if len(tlist) > 1 else -1
    total_omkt = tlist[2].text if len(tlist) > 2 else -1
    
    columns = [gk_oage, defense_oage, attack_oage, gk_mkt, defense_mkt, attack_mkt, gk_omkt, defense_omkt, 
               attack_omkt, total_oage, total_mkt, total_omkt]
    c =['gk_oage', 'defense_oage', 'attack_oage', 'gk_mkt', 'defense_mkt', 'attack_mkt', 'gk_omkt', 'defense_omkt', 
               'attack_omkt', 'total_oage', 'total_mkt', 'total_omkt']
    #columns = list(zip([gk_oage], [defense_oage], [attack_oage], [gk_mkt], [defense_mkt], [attack_mkt], [gk_omkt], 
    #                   [defense_omkt], [attack_omkt], [total_oage], [total_mkt], [total_omkt]))
    #df = pd.DataFrame(columns,
    #           columns =['gk_oage', 'defense_oage', 'attack_oage', 'gk_mkt', 'defense_mkt', 'attack_mkt', 'gk_omkt', 'defense_omkt', 
    #           'attack_omkt', 'total_oage', 'total_mkt', 'total_omkt'])
    #return df
    return columns, c

In [6]:
def get_transfer_data(link):
    response = s.get(link, headers=headers)
    content = response.content  
    site = BeautifulSoup(content, 'html.parser')
    
    rlist = site.find('div', class_='box viewport-tracking transfer-record').find('table').find('tbody').find_all('td')
    row1 = rlist[2]  if len(rlist)> 2 else -1
    row2 = rlist[5] if len(rlist)> 5 else -1
    r3list = site.find('div', class_='box viewport-tracking transfer-record').find('table').find('tfoot').find_all('td')
    row3 = r3list[1] if len(r3list)> 1 else -1
    
    income = row1.text.strip() if row1 != 0 else -1
    expenditure = row2.text.strip() if row2 != 0 else -1
    balance = row3.text.strip() if row3 != 0 else -1
    
    alist = site.find_all('table', class_='items')[0].find('tfoot').find_all('tr')
    arrival_sum = alist[0].find('td').text[4:].strip() if (len(alist)>0 and alist[0].find('td') is not None) else -1
    arrival_age = alist[1].find('td').text[12:].strip() if (len(alist)>1 and alist[1].find('td') is not None) else -1
    arrival_mkt = alist[2].find('td').text[32:].strip() if (len(alist)>2 and alist[2].find('td') is not None) else -1

    dlist = site.find_all('table', class_='items')[1].find('tfoot').find_all('tr')
    derpature_sum = dlist[0].find('td').text[4:].strip() if (len(dlist)>0 and dlist[0].find('td') is not None) else -1
    derpature_age = dlist[1].find('td').text[12:].strip() if (len(dlist)>1 and dlist[1].find('td') is not None) else -1
    derpature_mkt = dlist[2].find('td').text[34:].strip() if (len(dlist)>2 and dlist[2].find('td') is not None) else -1
    
    columns = [income,expenditure,balance,arrival_sum,arrival_age,arrival_mkt,derpature_sum,
              derpature_age,derpature_mkt]
    c = ['income','expenditure','balance','arrival_sum','arrival_age','arrival_mkt','derpature_sum',
              'derpature_age','derpature_mkt']
    #columns = list(zip([income],[expenditure],[balance],[arrival_sum],[arrival_age],[arrival_mkt],[derpature_sum],
    #          [derpature_age],[derpature_mkt]))
    #df = pd.DataFrame(columns,
    #               columns =['income','expenditure','balance','arrival_sum','arrival_age','arrival_mkt','derpature_sum',
    #          'derpature_age','derpature_mkt'])
    #return df
    return columns, c

In [7]:
def get_manager_data(link):
    response = s.get(link, headers=headers)
    content = response.content
    site = BeautifulSoup(content, 'html.parser')
    
    name = site.find('h1', class_='data-header__headline-wrapper').text.strip() if site.find('h1', class_='data-header__headline-wrapper') is not None else -1
    nlist = site.find('table',class_="auflistung").find_all('tr')
    age = -1
    avarage_time = -1
    for n in nlist:
        if n.find('th') is not None and n.find('th').text.strip() == "Age:":
            age =  n.find('td').text.strip()
        elif n.find('th') is not None and n.find('th').text.strip() == "Avg. term as coach:":
            avarage_time = n.find('td').text.strip()

    tlist = site.find('table',class_="items").find('tfoot').find_all('td')
    matches = tlist[1].text if len(tlist)>1 else -1
    wins =    tlist[2].text if len(tlist)>2 else -1
    draws =   tlist[3].text if len(tlist)>3 else -1
    losses =  tlist[4].text if len(tlist)>4 else -1
    points =  tlist[5].text if len(tlist)>5 else -1
    ppm =     tlist[6].text if len(tlist)>6 else -1

    teams = len(site.find('div',id='yw2').find('tbody').find_all('tr'))
    avarage_ppm = 0
    for i in site.find('div',id='yw2').find('tbody').find_all('tr'): 
        try: avarage_ppm+= float(i.find('td',class_="rechts").text.strip())
        except:continue
    avarage_ppm = avarage_ppm/teams if teams>0 else 0
    total_matches = 0
    for i in site.find('div',id='yw2').find('tbody').find_all('tr'): 
        try: total_matches+= int(i.find_all('td',class_="zentriert")[3].text.strip())
        except:continue
    
    ylist = site.find_all('div', class_="responsive-table")
    tlist = ylist[1].find('tfoot').find_all('td', class_="rechts") if len(ylist)>1 else []
    total_transfer = tlist[1].text if len(tlist)>1 else 0
    
    columns = [name, age, avarage_time, matches, wins, draws, losses, points, ppm, 
        teams, avarage_ppm, total_matches, total_transfer]
    c =['manager_name', 'manager_age', 'manager_avarage_time', 'manager_matches', 'manager_wins', 'manager_draws', 
        'manager_losses', 'manager_points', 'manager_ppm', 
               'manager_teams_coached', 'manager_avarage_ppm', 'manager_total_matches', 'manager_total_transfer']
    #columns = list(zip([name], [age], [avarage_time], [matches], [wins], [draws], [losses], [points], [ppm], 
    #       [teams], [avarage_ppm], [total_matches], [total_transfer]))
    #df = pd.DataFrame(columns,
    #               columns =['name', 'age', 'avarage_time', 'mmatches', 'mwins', 'mdraws', 'mlosses', 'mpoints', 'ppm', 
    #           'teams', 'avarage_ppm', 'total_matches', 'total_transfer'])
    #return df
    return columns, c

In [8]:
def get_all_club_data(link):
    response = s.get(link, headers=headers)
    content = response.content
    site = BeautifulSoup(content, 'html.parser')
    
    name = site.find('h1', class_='data-header__headline-wrapper data-header__headline-wrapper--oswald').text.strip()
    
    second_link = "https://www.transfermarkt.com" + site.find("a", class_="tm-subnav-item megamenu")['href']
    response = requests.get(second_link, headers=headers)
    content = response.content
    site = BeautifulSoup(content, 'html.parser')
    
    squad_link = "https://www.transfermarkt.com" + site.find_all("a", class_="content-link", title=name)[0]['href']
    transfer_link = "https://www.transfermarkt.com" + site.find_all("a", class_="content-link", title=name)[3]['href']
    manager_link = "https://www.transfermarkt.com" + site.find('div', class_='box viewport-tracking').find('tr', style="background-color: #FBFBFB;").find('a')['href']
    
    data1, c1 = get_stats_data(link)
    data2, c2 = get_squad_data(squad_link)
    data3, c3 = get_transfer_data(transfer_link)
    data4, c4 = get_manager_data(manager_link)
    
    return data1+data2+data3+data4, c1+c2+c3+c4

In [9]:
def get_year_before_link(link):
    year = int(link[-4:])
    year -= 1
    link = link[:-4] + str(year)
    return link

In [10]:
def scrapping_accuracy(my_df):
    count = my_df.isin(['-1']).sum(axis=0)
    total = my_df.shape[0]*my_df.shape[1]
    invalid_p = count/total
    return invalid_p

In [11]:
def get_table(link):
    response = s.get(link, headers=headers, verify=False)
    content = response.content
    site = BeautifulSoup(content, 'html.parser')
    
    table = site.find("table", class_="items")
    rows = table.find_all("tr")[1:]

    links = [row.find("td", class_="no-border-links hauptlink").find("a")['href'] for row in rows]

    teams = [row.find("td", class_="no-border-links hauptlink").find("a").text for row in rows]
    games = [row.find_all("td", class_="zentriert")[1].text for row in rows]
    wins = [row.find_all("td", class_="zentriert")[2].text for row in rows]
    draws = [row.find_all("td", class_="zentriert")[3].text for row in rows]
    losses = [row.find_all("td", class_="zentriert")[4].text for row in rows]
    goals = [row.find_all("td", class_="zentriert")[5].text for row in rows]
    saldo = [row.find_all("td", class_="zentriert")[6].text for row in rows]
    pts = [row.find_all("td", class_="zentriert")[7].text for row in rows]
    rank = [i for i in range(1,21)]
    
    columns = list(zip(teams, games, wins, draws, losses, goals, saldo, pts, rank))

    df_table = pd.DataFrame(columns,
                   columns =['Name', 'Games', 'Wins', 'Draws', 'Losses', 'Goals', 'Balance', 'Points', 'Final Position'])
    
    clubs_data = []
    print("Season: "+ links[0][-4:])
    for i,link_ in enumerate(links):
        sleep(8)
        print(str(i+1) + " " + teams[i], 'https://www.transfermarkt.com'+ get_year_before_link(link_))
        try:
            cd, c = get_all_club_data('https://www.transfermarkt.com'+ get_year_before_link(link_))
            clubs_data.append(cd)
        except: 
            clubs_data.append(([-1]*40))
            print('error')
            continue
        
    df_clubs_data = pd.DataFrame(clubs_data, columns=c)
    
    final_df = pd.concat([df_table, df_clubs_data], axis = 1)
    
    #print("Invalid values percentage: " + str(scrapping_accuracy(final_df)))
    
    #return pd.concat([df_table, df_clubs_data], axis = 1), df_table, df_clubs_data, clubs_data
    return final_df

In [12]:
for n in range (2000,2023):
    filename = "pl-"+str(n)+".csv"
    link = "https://www.transfermarkt.com/premier-league/tabelle/wettbewerb/GB1/saison_id/" + str(n)
    df = get_table(link)
    df.to_csv(filename, encoding='utf-8', index=False)

Season: 2000
1 Man Utd https://www.transfermarkt.com/manchester-united/spielplan/verein/985/saison_id/1999
2 Arsenal https://www.transfermarkt.com/fc-arsenal/spielplan/verein/11/saison_id/1999
3 Liverpool https://www.transfermarkt.com/fc-liverpool/spielplan/verein/31/saison_id/1999
4 Leeds https://www.transfermarkt.com/leeds-united/spielplan/verein/399/saison_id/1999
5 Ipswich https://www.transfermarkt.com/ipswich-town/spielplan/verein/677/saison_id/1999
error
6 Chelsea https://www.transfermarkt.com/fc-chelsea/spielplan/verein/631/saison_id/1999
7 Sunderland https://www.transfermarkt.com/afc-sunderland/spielplan/verein/289/saison_id/1999
8 Aston Villa https://www.transfermarkt.com/aston-villa/spielplan/verein/405/saison_id/1999
9 Charlton https://www.transfermarkt.com/charlton-athletic/spielplan/verein/358/saison_id/1999
10 Southampton https://www.transfermarkt.com/fc-southampton/spielplan/verein/180/saison_id/1999
11 Newcastle https://www.transfermarkt.com/newcastle-united/spielplan/v

13 Birmingham https://www.transfermarkt.com/birmingham-city/spielplan/verein/337/saison_id/2001
14 Fulham https://www.transfermarkt.com/fc-fulham/spielplan/verein/931/saison_id/2001
15 Leeds https://www.transfermarkt.com/leeds-united/spielplan/verein/399/saison_id/2001
16 Aston Villa https://www.transfermarkt.com/aston-villa/spielplan/verein/405/saison_id/2001
17 Bolton https://www.transfermarkt.com/bolton-wanderers/spielplan/verein/355/saison_id/2001
18 West Ham https://www.transfermarkt.com/west-ham-united/spielplan/verein/379/saison_id/2001
19 West Brom https://www.transfermarkt.com/west-bromwich-albion/spielplan/verein/984/saison_id/2001
20 Sunderland https://www.transfermarkt.com/afc-sunderland/spielplan/verein/289/saison_id/2001
Invalid values percentage: Name                      0.00000
Games                     0.00000
Wins                      0.00000
Draws                     0.00000
Losses                    0.00000
Goals                     0.00000
Balance                 

Season: 2005
1 Chelsea https://www.transfermarkt.com/fc-chelsea/spielplan/verein/631/saison_id/2004
2 Man Utd https://www.transfermarkt.com/manchester-united/spielplan/verein/985/saison_id/2004
3 Liverpool https://www.transfermarkt.com/fc-liverpool/spielplan/verein/31/saison_id/2004
4 Arsenal https://www.transfermarkt.com/fc-arsenal/spielplan/verein/11/saison_id/2004
5 Tottenham https://www.transfermarkt.com/tottenham-hotspur/spielplan/verein/148/saison_id/2004
6 Blackburn https://www.transfermarkt.com/blackburn-rovers/spielplan/verein/164/saison_id/2004
7 Newcastle https://www.transfermarkt.com/newcastle-united/spielplan/verein/762/saison_id/2004
8 Bolton https://www.transfermarkt.com/bolton-wanderers/spielplan/verein/355/saison_id/2004
9 West Ham https://www.transfermarkt.com/west-ham-united/spielplan/verein/379/saison_id/2004
10 Wigan https://www.transfermarkt.com/wigan-athletic/spielplan/verein/1071/saison_id/2004
11 Everton https://www.transfermarkt.com/fc-everton/spielplan/verein

16 Bolton https://www.transfermarkt.com/bolton-wanderers/spielplan/verein/355/saison_id/2006
17 Fulham https://www.transfermarkt.com/fc-fulham/spielplan/verein/931/saison_id/2006
18 Reading https://www.transfermarkt.com/fc-reading/spielplan/verein/1032/saison_id/2006
19 Birmingham https://www.transfermarkt.com/birmingham-city/spielplan/verein/337/saison_id/2006
20 Derby https://www.transfermarkt.com/derby-county/spielplan/verein/22/saison_id/2006
Invalid values percentage: Name                      0.0
Games                     0.0
Wins                      0.0
Draws                     0.0
Losses                    0.0
Goals                     0.0
Balance                   0.0
Points                    0.0
Final Position            0.0
name                      0.0
matches                   0.0
wins                      0.0
draws                     0.0
losses                    0.0
pts                       0.0
goals                     0.0
spectators                0.0
gk_oage     

Season: 2010
1 Man Utd https://www.transfermarkt.com/manchester-united/spielplan/verein/985/saison_id/2009
2 Chelsea https://www.transfermarkt.com/fc-chelsea/spielplan/verein/631/saison_id/2009
3 Man City https://www.transfermarkt.com/manchester-city/spielplan/verein/281/saison_id/2009
4 Arsenal https://www.transfermarkt.com/fc-arsenal/spielplan/verein/11/saison_id/2009
5 Tottenham https://www.transfermarkt.com/tottenham-hotspur/spielplan/verein/148/saison_id/2009
6 Liverpool https://www.transfermarkt.com/fc-liverpool/spielplan/verein/31/saison_id/2009
7 Everton https://www.transfermarkt.com/fc-everton/spielplan/verein/29/saison_id/2009
8 Fulham https://www.transfermarkt.com/fc-fulham/spielplan/verein/931/saison_id/2009
9 Aston Villa https://www.transfermarkt.com/aston-villa/spielplan/verein/405/saison_id/2009
10 Sunderland https://www.transfermarkt.com/afc-sunderland/spielplan/verein/289/saison_id/2009
11 West Brom https://www.transfermarkt.com/west-bromwich-albion/spielplan/verein/98

11 Norwich https://www.transfermarkt.com/norwich-city/spielplan/verein/1123/saison_id/2011
12 Fulham https://www.transfermarkt.com/fc-fulham/spielplan/verein/931/saison_id/2011
13 Stoke City https://www.transfermarkt.com/stoke-city/spielplan/verein/512/saison_id/2011
14 Southampton https://www.transfermarkt.com/fc-southampton/spielplan/verein/180/saison_id/2011
15 Aston Villa https://www.transfermarkt.com/aston-villa/spielplan/verein/405/saison_id/2011
16 Newcastle https://www.transfermarkt.com/newcastle-united/spielplan/verein/762/saison_id/2011
17 Sunderland https://www.transfermarkt.com/afc-sunderland/spielplan/verein/289/saison_id/2011
18 Wigan https://www.transfermarkt.com/wigan-athletic/spielplan/verein/1071/saison_id/2011
19 Reading https://www.transfermarkt.com/fc-reading/spielplan/verein/1032/saison_id/2011
20 QPR https://www.transfermarkt.com/queens-park-rangers/spielplan/verein/1039/saison_id/2011
Invalid values percentage: Name                      0.0
Games                

Season: 2015
1 Leicester https://www.transfermarkt.com/leicester-city/spielplan/verein/1003/saison_id/2014
2 Arsenal https://www.transfermarkt.com/fc-arsenal/spielplan/verein/11/saison_id/2014
3 Tottenham https://www.transfermarkt.com/tottenham-hotspur/spielplan/verein/148/saison_id/2014
4 Man City https://www.transfermarkt.com/manchester-city/spielplan/verein/281/saison_id/2014
5 Man Utd https://www.transfermarkt.com/manchester-united/spielplan/verein/985/saison_id/2014
6 Southampton https://www.transfermarkt.com/fc-southampton/spielplan/verein/180/saison_id/2014
7 West Ham https://www.transfermarkt.com/west-ham-united/spielplan/verein/379/saison_id/2014
8 Liverpool https://www.transfermarkt.com/fc-liverpool/spielplan/verein/31/saison_id/2014
9 Stoke City https://www.transfermarkt.com/stoke-city/spielplan/verein/512/saison_id/2014
10 Chelsea https://www.transfermarkt.com/fc-chelsea/spielplan/verein/631/saison_id/2014
11 Everton https://www.transfermarkt.com/fc-everton/spielplan/verein

16 Huddersfield https://www.transfermarkt.com/huddersfield-town/spielplan/verein/1110/saison_id/2016
17 Southampton https://www.transfermarkt.com/fc-southampton/spielplan/verein/180/saison_id/2016
18 Swansea https://www.transfermarkt.com/swansea-city/spielplan/verein/2288/saison_id/2016
19 Stoke City https://www.transfermarkt.com/stoke-city/spielplan/verein/512/saison_id/2016
20 West Brom https://www.transfermarkt.com/west-bromwich-albion/spielplan/verein/984/saison_id/2016
Invalid values percentage: Name                      0.0
Games                     0.0
Wins                      0.0
Draws                     0.0
Losses                    0.0
Goals                     0.0
Balance                   0.0
Points                    0.0
Final Position            0.0
name                      0.0
matches                   0.0
wins                      0.0
draws                     0.0
losses                    0.0
pts                       0.0
goals                     0.0
spectators    

Season: 2020
1 Man City https://www.transfermarkt.com/manchester-city/spielplan/verein/281/saison_id/2019
2 Man Utd https://www.transfermarkt.com/manchester-united/spielplan/verein/985/saison_id/2019
3 Liverpool https://www.transfermarkt.com/fc-liverpool/spielplan/verein/31/saison_id/2019
4 Chelsea https://www.transfermarkt.com/fc-chelsea/spielplan/verein/631/saison_id/2019
5 Leicester https://www.transfermarkt.com/leicester-city/spielplan/verein/1003/saison_id/2019
6 West Ham https://www.transfermarkt.com/west-ham-united/spielplan/verein/379/saison_id/2019
7 Tottenham https://www.transfermarkt.com/tottenham-hotspur/spielplan/verein/148/saison_id/2019
8 Arsenal https://www.transfermarkt.com/fc-arsenal/spielplan/verein/11/saison_id/2019
9 Leeds https://www.transfermarkt.com/leeds-united/spielplan/verein/399/saison_id/2019
10 Everton https://www.transfermarkt.com/fc-everton/spielplan/verein/29/saison_id/2019
11 Aston Villa https://www.transfermarkt.com/aston-villa/spielplan/verein/405/sa

13 Wolves https://www.transfermarkt.com/wolverhampton-wanderers/spielplan/verein/543/saison_id/2021
14 West Ham https://www.transfermarkt.com/west-ham-united/spielplan/verein/379/saison_id/2021
15 Bournemouth https://www.transfermarkt.com/afc-bournemouth/spielplan/verein/989/saison_id/2021
16 Nottm Forest https://www.transfermarkt.com/nottingham-forest/spielplan/verein/703/saison_id/2021
17 Everton https://www.transfermarkt.com/fc-everton/spielplan/verein/29/saison_id/2021
18 Leicester https://www.transfermarkt.com/leicester-city/spielplan/verein/1003/saison_id/2021
19 Leeds https://www.transfermarkt.com/leeds-united/spielplan/verein/399/saison_id/2021
20 Southampton https://www.transfermarkt.com/fc-southampton/spielplan/verein/180/saison_id/2021
Invalid values percentage: Name                      0.0
Games                     0.0
Wins                      0.0
Draws                     0.0
Losses                    0.0
Goals                     0.0
Balance                   0.0
Points